In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from geopy.geocoders import Nominatim
import pandas as pd
import time

In [2]:
ciudad = "madrid"
url = 'https://www.spain.info/es/resultados-busqueda/index.html?q='
url2 = '&reloaded&page=1&rpp=36&subTypeFilter=Museo,Monumento,Jard%C3%ADn%20hist%C3%B3rico'

In [3]:
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)

opciones.headless=False    # si True, no aperece la ventana (headless=no visible)

opciones.add_argument('--start-maximized')         # comienza maximizado

opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies

opciones.add_argument('--incognito')              # incognito


PATH='./chromedriver'
driver=webdriver.Chrome(PATH, options=opciones)
driver.get(url + ciudad + url2)

In [4]:
texto_visitar = driver.find_elements_by_class_name('title')

In [5]:
frase = []
for i in texto_visitar:
    frase.append(i.text)

frase = frase[11:]
frase

['CaixaForum Madrid',
 'Museo del Real Observatorio de Madrid',
 'Real Colegiata de San Isidro de Madrid',
 'Museo Arqueológico Regional de la Comunidad de Madrid',
 'Presidencia de la Comunidad de Madrid (Antigua Casa de Correos)',
 'Museo Nacional del Prado']

In [6]:
'''
direccion = []
for i in range(0,len(frase)):
    geolocator = Nominatim(user_agent="edu")
    location = geolocator.geocode(frase[i])
    direccion.append(location.address)
    time.sleep(3)
'''

'\ndireccion = []\nfor i in range(0,len(frase)):\n    geolocator = Nominatim(user_agent="edu")\n    location = geolocator.geocode(frase[i])\n    direccion.append(location.address)\n    time.sleep(3)\n'

In [7]:
'''
direccione = []
for i in direccion:
    corte = i.split(",")[1:]
    direccione.append(corte)
'''

'\ndireccione = []\nfor i in direccion:\n    corte = i.split(",")[1:]\n    direccione.append(corte)\n'

In [8]:
driver.get('https://www.spain.info/es/resultados-busqueda/index.html?q=barcelona&reloaded&page=1&rpp=36&typeFilter=SEG-TUR-AtractivoTuristicoNatural')

In [9]:
texto_naturaleza = driver.find_elements_by_class_name('title')

frase2 = []
for i in texto_naturaleza:
    frase2.append(i.text)

In [10]:
frase2 = frase2[11:17]
frase2

['Parque Natural de la Montaña de Montserrat',
 "Parque Natural de Sant Llorenç del Munt i serra de l'Obac",
 'Parque Natural de las Fuentes Carrionas y Fuente Cobre-Montaña Palentina',
 'Parque Natural de Serres de Cadí-Moixeró',
 'Parque Natural del Montseny']

In [12]:
df = pd.DataFrame(list(zip(frase)), columns = ['Visitar'])
df

,Visitar
0,CaixaForum Madrid
1,Museo del Real Observatorio de Madrid
2,Real Colegiata de San Isidro de Madrid
3,Museo Arqueológico Regional de la Comunidad de...
4,Presidencia de la Comunidad de Madrid (Antigua...
5,Museo Nacional del Prado


In [13]:
df2 = pd.DataFrame(list(zip(frase2)), columns = ['Naturaleza'])
df2

,Naturaleza
0,Parque Natural de la Montaña de Montserrat
1,Parque Natural de Sant Llorenç del Munt i serr...
2,Parque Natural de las Fuentes Carrionas y Fuen...
3,Parque Natural de Serres de Cadí-Moixeró
4,Parque Natural del Montseny


In [14]:
result = pd.concat([df, df2], axis=1)

In [15]:
result

,Visitar,Naturaleza
0,CaixaForum Madrid,Parque Natural de la Montaña de Montserrat
1,Museo del Real Observatorio de Madrid,Parque Natural de Sant Llorenç del Munt i serr...
2,Real Colegiata de San Isidro de Madrid,Parque Natural de las Fuentes Carrionas y Fuen...
3,Museo Arqueológico Regional de la Comunidad de...,Parque Natural de Serres de Cadí-Moixeró
4,Presidencia de la Comunidad de Madrid (Antigua...,Parque Natural del Montseny
5,Museo Nacional del Prado,NaN
